In [1]:
!pip install pandas
!pip install numpy

In [2]:
import pandas as pd
import numpy as np
import warnings;warnings.filterwarnings('ignore')

In [18]:
problem = pd.read_csv('problem.csv')
user = pd.read_csv('user.csv')
target_problem = "1000"

In [19]:
print(problem)

문제번호                                    링크           난이도       제목  \
0      1000  https://www.acmicpc.net/problem/1000      Bronze V      A+B   
1      1001  https://www.acmicpc.net/problem/1001      Bronze V      A-B   
2      1007  https://www.acmicpc.net/problem/1007       Gold II    벡터 매칭   
3      1008  https://www.acmicpc.net/problem/1008     Bronze IV      A/B   
4      1009  https://www.acmicpc.net/problem/1009    Bronze III     분산처리   
...     ...                                   ...           ...      ...   
18197  2549  https://www.acmicpc.net/problem/2549        Gold I  루빅의 사각형   
18198  2794  https://www.acmicpc.net/problem/2794    Platinum I       화성   
18199  3110  https://www.acmicpc.net/problem/3110      Gold III      부등식   
18200  3148  https://www.acmicpc.net/problem/3148  Platinum III       빙고   
18201  4160  https://www.acmicpc.net/problem/4160    Platinum V       이혼   

             분류     해결  평균시도  
0            수학  98776  2.26  
1            수학  81082  1.39  
2

In [25]:
pd.set_option('max_colwidth',100)
#각각의 장르와 그 장르의 id를 추출해 낼 수 있다
problem_df[['분류','난이도']][:1]

,분류,난이도
0,수학,Bronze V


In [26]:
problem_df = problem[['문제번호','난이도','분류','해결','평균시도']]
print(problem_df)

문제번호           난이도        분류     해결  평균시도
0      1000      Bronze V        수학  98776  2.26
1      1001      Bronze V        수학  81082  1.39
2      1007       Gold II        수학    858  2.72
3      1008     Bronze IV        수학  53039  3.01
4      1009    Bronze III        수학   6579  3.82
...     ...           ...       ...    ...   ...
18197  2549        Gold I  중간에서 만나기    234  3.18
18198  2794    Platinum I  중간에서 만나기     11  3.00
18199  3110      Gold III  중간에서 만나기     38  2.79
18200  3148  Platinum III  중간에서 만나기      3  4.00
18201  4160    Platinum V  중간에서 만나기      9  2.67

[18202 rows x 5 columns]


In [28]:
#literal_eval() 함수를 사용하면 이 문자열을 문자열이 의미하는 list[dict1, dict2] 객체로 만들 수 있다
from ast import literal_eval
problem_df['분류'] = problem_df['분류'].apply(literal_eval) #genres 칼럼은 문자열이 아닌 실제 리스트 내부에 여러 장르 딕셔너리로 구성된 객체들을 가진다
problem_df['난이도'] = problem_df['난이도'].apply(literal_eval)


ValueError: malformed node or string: <_ast.Name object at 0x000001FC8452A160>

In [96]:
from sklearn.feature_extraction.text import CountVectorizer

#CountVectorizer를 적용하기 위해 공백문자로 word 단위가 구분되는 문자열로 변환
count_vect  = CountVectorizer(min_df = 0, ngram_range=(1,2))
genre_mat = count_vect.fit_transform(problem_df['분류'])
print(genre_mat.shape)

(18202, 151)


In [91]:
from sklearn.metrics.pairwise import cosine_similarity

genre_sim = cosine_similarity(genre_mat, genre_mat)
level_sim = cosine_similarity(level_mat,level_mat)
print(genre_sim.shape)
print(genre_sim)
print(level_sim)

(18202, 18202)
[[1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 [1. 1. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 1. 1.]
 [0. 0. 0. ... 1. 1. 1.]
 [0. 0. 0. ... 1. 1. 1.]]
[[1.         1.         0.         ... 0.         0.         0.        ]
 [1.         1.         0.         ... 0.         0.         0.        ]
 [0.         0.         1.         ... 0.33333333 0.         0.        ]
 ...
 [0.         0.         0.33333333 ... 1.         0.33333333 0.        ]
 [0.         0.         0.         ... 0.33333333 1.         0.57735027]
 [0.         0.         0.         ... 0.         0.57735027 1.        ]]


In [92]:
genre_sim_sorted_ind = genre_sim.argsort()[:,::-1]
print(genre_sim_sorted_ind[:1])
level_sim_sorted_ind = level_sim.argsort()[:,::-1]
print(level_sim_sorted_ind[:1])


[[    0  1142  1140 ... 12126 12125  9100]]
[[    0   897   985 ... 11843 11842  9100]]


In [97]:
#장르 유사도에 따라 영화를 추천하는 함수
#함수명 : find_sim_movie()
#movies_df DataFrame : 기반 데이터
#genre_sim_sorted_ind : 레코드별 장르 코사인 유사도 인덱스
#DataFrame : 고객이 선정한 추천 기준이 되는 영화제목, 추천할 영화 건수를 입력하면 추천 영화 정보를 가짐
def find_sim_problem(df, sorted_ind, title_name, top_n = 10):
    
    title_problem = df[df['문제번호']== int(title_name)]
    
    title_index = title_problem.index.values
    similar_indexes = sorted_ind[title_index,:(top_n)]
    
    similar_indexes = similar_indexes.reshape(-1)
    
    return df.iloc[similar_indexes]

In [99]:
similar_movies = find_sim_problem(problem_df, genre_sim_sorted_ind,target_problem, 40)
similar_movies[['문제번호','평균시도','해결','난이도','분류']]

,문제번호,평균시도,해결,난이도,분류
0,1000,2.26,98776,Bronze V,수학
1142,1193,1.88,14902,Bronze II,수학
1140,1187,2.32,34,Diamond V,수학
1139,1183,2.21,43,Silver V,수학
1138,1179,2.84,85,Gold III,수학
...,...,...,...,...,...
12584,5533,1.39,1263,Bronze I,사칙연산
12583,5532,1.56,3586,Bronze IV,사칙연산
12582,5522,1.12,2621,Bronze V,사칙연산
12581,5361,1.52,1583,Bronze III,사칙연산


In [86]:
problem_df[['문제번호','난이도','평균시도']].sort_values('평균시도',ascending=True)[:10]



,문제번호,난이도,평균시도
15157,16371,Diamond V,1.0
17750,18350,Diamond I,1.0
12830,3858,Platinum I,1.0
17748,18163,Ruby II,1.0
17747,17879,Platinum II,1.0
16392,19327,Diamond IV,1.0
13547,6752,Silver V,1.0
12832,5041,Platinum II,1.0
16109,2214,Gold III,1.0
14659,11373,Platinum III,1.0


In [159]:
#난이도를 정제하여 난이도 수준을 맞춤
level_array = ['Bronze','Silver','Gold','Platium','Diamond','Ruby']
print(level_array[0]) 
def level_set(df,target_problem):
    level_idx = df.index[df['문제번호']==int(target_problem)]
    level = df.loc[level_idx[0]]['난이도'].split(' ')[0]
    df['난이도 정제'] = df['난이도'].str.split(' ')
    return df['난이도 정제'].str.contains('Bronze')
return_df = level_set(problem_df,target_problem)
print(return_df)

Bronze
0       NaN
1       NaN
2       NaN
3       NaN
4       NaN
         ..
18197   NaN
18198   NaN
18199   NaN
18200   NaN
18201   NaN
Name: 난이도 정제, Length: 18202, dtype: float64


In [89]:
C = problem_df['평균시도'].mean()
m = problem_df['해결'].quantile(0.6)
print('C: ',round(C,3),'m: ',round(m,3))

C:  3.205 m:  229.0


In [ ]:
#원래 점수에 
percentile = 0.6
m = problem_df['해결'].quantile(0.6)
C = problem_df['평균시도'].mean()

def weighted_vote_average(record):
    v = record['해결']
    R = record['평균시도']

    return  (v/(v+m)) * R + (v/(v+m)) * C


movies['weighted_vote'] = movies.apply(weighted_vote_average,axis=1)